In [1]:
import pandas as pd  
import numpy as np 

import matplotlib.pyplot as plt 
import seaborn as sns 

In [18]:
df = pd.read_excel("UK_Data.xlsx")

In [20]:
df.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Amount
0,0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333828 entries, 0 to 333827
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Unnamed: 0   333828 non-null  int64         
 1   InvoiceNo    333828 non-null  int64         
 2   StockCode    333828 non-null  object        
 3   Description  333828 non-null  object        
 4   Quantity     333828 non-null  int64         
 5   InvoiceDate  333828 non-null  datetime64[ns]
 6   UnitPrice    333828 non-null  float64       
 7   CustomerID   333828 non-null  int64         
 8   Country      333828 non-null  object        
 9   Amount       333828 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(4), object(3)
memory usage: 25.5+ MB


In [33]:
df['InvoiceDate'] = df['InvoiceDate'].dt.date

In [34]:
ref = df['InvoiceDate'].max()



In [5]:
df['InvoiceDate'] = df['InvoiceDate'].dt.date

In [50]:
new = df.loc[df.groupby(df['CustomerID'])['InvoiceDate'].idxmax()][['CustomerID', 'InvoiceDate']]
new['InvoiceDate'] = pd.to_datetime(new['InvoiceDate'])
new.head()

,CustomerID,InvoiceDate
330694,12747,2011-12-07
333774,12748,2011-12-09
327472,12749,2011-12-06
329246,12820,2011-12-06
105304,12821,2011-05-09


In [51]:
new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3818 entries, 330694 to 257930
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   CustomerID   3818 non-null   int64         
 1   InvoiceDate  3818 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1)
memory usage: 89.5 KB


In [54]:
ref = pd.to_datetime(ref)

In [57]:
new['recency'] = (ref - new['InvoiceDate']).dt.days


In [59]:
new.reset_index(drop = True)

,CustomerID,InvoiceDate,recency
0,12747,2011-12-07,2
1,12748,2011-12-09,0
2,12749,2011-12-06,3
3,12820,2011-12-06,3
4,12821,2011-05-09,214
...,...,...,...
3813,18280,2011-03-07,277
3814,18281,2011-06-12,180
3815,18282,2011-12-02,7
3816,18283,2011-12-06,3
